# Options Chains Analysis

Version 3.1 of the OpenBB SDK provides a suite of enhancements to the Options module.  This notebook will highlight the changes and demonstrate how to get started using the new tools.

## What's Changed

- There's a new way to  `load`
  
        `openbb.stocks.options.load_options_chains()`

    The legacy method (`openbb.stocks.options.chains()`) remains in service, but some minor changes to existing scripts will be required to take advantage of the new functionality.  Don't worry, it's nothing that `Find & Replace` can't easily handle.  Let's compare below.

In [2]:
# Select the `obb` Python environment as the session's kernel, then import the OpenBB SDK.

from openbb_terminal.sdk import openbb
import pandas as pd
from openbb_terminal.economy.plot_view import show_plot

In [4]:
# The legacy method

spy_chains = openbb.stocks.options.chains("SPY")
spy_chains.head(2)

,optionType,expiration,strike,lastPrice,bid,ask,openInterest,volume
0,call,2023-06-23,275.0,158.44,159.31,159.45,0,2
1,put,2023-06-23,275.0,0.01,0.00,0.01,1007,1


In [5]:
# The new method

spy = openbb.stocks.options.load_options_chains("SPY", "Nasdaq")
spy.chains.head(2)

,optionType,expiration,strike,lastPrice,bid,ask,openInterest,volume,dte
0,call,2023-06-23,275.0,158.44,159.31,159.45,0,2,0
1,put,2023-06-23,275.0,0.01,0.00,0.01,1007,1,0


The same response (the options chains) is in both variables, but the new method contains more than just the chains data previously fetched.

In [6]:
spy.__dict__.keys()

dict_keys(['source', 'SYMBOLS', 'symbol', 'chains', 'expirations', 'strikes', 'underlying_price', 'last_price', 'underlying_name', 'hasIV', 'hasGreeks', 'get_available_greeks', 'date'])

The new object eliminates the need to make an additional request for the price of the underlying asset, or the list of expirations and strikes.

In [7]:
# Instead of:

last_price = openbb.stocks.options.price("SPY")

last_price

434.3

In [8]:
# Use the Options data object:

spy.last_price

434.3

- There are new sources of data

    Two additional data sources have been added, `CBOE` and `TMX`, with the latter covering the Canadian market.  The new default source is `CBOE`.  The DataFrame has been transposed below to better illustrate the additional columns available by default.

In [9]:
spy = openbb.stocks.options.load_options_chains("SPY")

spy.chains.query("`strike` == 435").head(6).transpose()

,186,187,456,457,636,637
expiration,2023-06-23,2023-06-23,2023-06-26,2023-06-26,2023-06-27,2023-06-27
strike,435.0,435.0,435.0,435.0,435.0,435.0
optionType,call,put,call,put,call,put
contractSymbol,SPY230623C00435000,SPY230623P00435000,SPY230626C00435000,SPY230626P00435000,SPY230627C00435000,SPY230627P00435000
bid,0.28,0.89,0.95,1.51,1.36,1.86
bidSize,829,211,112,58,97,113
ask,0.29,0.9,0.96,1.52,1.37,1.87
askSize,387,11,201,150,168,150
impliedVolatility,0.145,0.148,0.0743,0.0748,0.0863,0.0862
openInterest,27779,24957,7187,8872,1820,5341


`TMX` provides historical EOD chains data, beginning in 2009.  Notably, this data contains implied volatility, the current quotes do not.

In [10]:
xiu = openbb.stocks.options.load_options_chains("XIU", "TMX", "2009-07-01")
print("The closing price of ", xiu.symbol,  " on, ", xiu.date, ", was: $", xiu.last_price, sep ="")
xiu.chains.query("`strike` == 15").head(6).transpose()

The closing price of XIU on, 2009-07-02, was: $15.59


,10,11,30,31,64,65
expiration,2009-07-17 00:00:00,2009-07-17 00:00:00,2009-08-21 00:00:00,2009-08-21 00:00:00,2009-09-18 00:00:00,2009-09-18 00:00:00
strike,15.0,15.0,15.0,15.0,15.0,15.0
optionType,call,put,call,put,call,put
date,2009-07-02,2009-07-02,2009-07-02,2009-07-02,2009-07-02,2009-07-02
bid,0.68,0.1,0.94,0.34,1.14,0.56
ask,0.77,0.15,1.05,0.43,1.26,0.63
bidSize,510,500,510,800,510,510
askSize,52,2,330,300,300,300
lastPrice,0.77,0.15,1.05,0.43,1.26,0.63
volume,0,60,0,0,0,15


In [11]:
xiu = openbb.stocks.options.load_options_chains("XIU", "TMX")
xiu.chains.query("30 <= `strike` <= 31").head(10)

,expiration,strike,optionType,bid,ask,lastPrice,change,openInterest,volume,dte
14,2023-06-23,30.00,call,0.00,0.05,0.07,0.0,30,0,0
15,2023-06-23,30.00,put,0.22,0.55,0.28,0.0,4,0,0
16,2023-06-23,30.25,call,0.00,0.05,0.05,0.0,0,0,0
17,2023-06-23,30.25,put,0.47,0.75,0.64,0.0,0,0,0
18,2023-06-23,30.50,call,0.00,0.05,0.05,0.0,49,0,0
19,2023-06-23,30.50,put,0.72,1.00,0.89,0.0,0,0,0
20,2023-06-23,30.75,call,0.00,0.05,0.05,0.0,51,0,0
21,2023-06-23,30.75,put,0.97,1.25,1.14,0.0,0,0,0
22,2023-06-23,31.00,call,0.00,0.05,0.05,0.0,0,0,0
23,2023-06-23,31.00,put,1.22,1.50,1.39,0.0,0,0,0


- Methods are bound to the returned object that sort and filter the data
    Along with attributes listed above - `spy.__dict__.keys()` - methods are provided for extracting key insights from the raw data.

    - `get_stats()`
    - `get_skew()`
    - `get_straddle()`
    - `get_strangle()`
    - `get_vertical_call_spread()`
    - `get_vertical_put_spread()`
    - `get_strategies()`

`impliedVolatility` is a required field for `get_skew()`; therefore, only sources returning `hasIV` as `True` will work.  All other callables are compatible with every source.


## Getting Started

Assign a ticker to a new variable.

In [12]:
ticker = openbb.stocks.options.load_options_chains("QQQ")
print(ticker.underlying_name, '\n', "Symbol: ", ticker.symbol, '\n', "Source: ", ticker.source, '\n', "Last Price: " , ticker.last_price, sep = "")

INVESCO QQQ TR UNIT SER 1
Symbol: QQQ
Source: CBOE
Last Price: 364.02


Let's look at volume and open interest, by expiration.

In [13]:
oi = ticker.get_stats()[["Puts OI", "Calls OI"]]
oi["Puts OI"] = oi["Puts OI"] * (-1)
vol = ticker.get_stats()[["Puts Volume", "Calls Volume"]]
vol["Puts Volume"] = vol["Puts Volume"] * (-1)

In [14]:
openbb.forecast.plot(oi, oi.columns.to_list())

In [15]:
openbb.forecast.plot(vol, vol.columns.to_list())

The concentration of volume is clearly at the front contract, while the open interest paints a different picture.  The same analysis can be easily applied by strike.


In [16]:
strike_oi = ticker.get_stats("strike")[["Puts OI", "Calls OI"]]
strike_oi["Puts OI"] = strike_oi["Puts OI"] * (-1)
strike_vol = ticker.get_stats("strike")[["Puts Volume", "Calls Volume"]]
strike_vol["Puts Volume"] = strike_vol["Puts Volume"] * (-1)

In [17]:
openbb.forecast.plot(strike_oi, strike_oi.columns.to_list())

In [18]:
openbb.forecast.plot(strike_vol, strike_vol.columns.to_list())

Delta exposure can easily be calculated by multiplying the delta value in the chains data by (OI * 100 * share price)

In [19]:
df = ticker.chains[["expiration", "dte", "strike", "optionType", "openInterest", "delta"]]
df["Delta Exposure"] = df["delta"] * df["openInterest"] * (100) * ticker.last_price
dex = pd.DataFrame()
dex["Call DEX"] = df.query("`optionType` == 'call'").set_index(["expiration","strike"])[["Delta Exposure"]]
dex["Put DEX"] = df.query("`optionType` == 'put'").set_index(["expiration","strike"])[["Delta Exposure"]]

# By expiration date

dex_by_expiry = dex.reset_index().groupby("expiration")[["Call DEX", "Put DEX"]].sum()

openbb.forecast.plot(dex_by_expiry, dex_by_expiry.columns.to_list())

In [20]:
# By strike price

dex_by_strike = dex.reset_index().groupby("strike")[["Call DEX", "Put DEX"]].sum()

openbb.forecast.plot(dex_by_strike, dex_by_strike.columns.to_list())

The Put/Call Ratios can be visualized using columns returned by, `get_stats()`

In [21]:
itm = ticker.get_stats()[["ITM Percent"]]
pcr = ticker.get_stats()[["OI Ratio"]]
pcr_fig = show_plot(itm, pcr, external_axes = True)
pcr_fig.update_layout(
    xaxis = {'title': 'Expiration Date'},
    yaxis= {'title': '% of OI ITM'},
    yaxis2 = {'title': 'Put/Call Ratio'},
    title =  f"{ticker.symbol} - Put/Call Ratio vs % ITM",
    title_y = 0.98,
    title_x = 0.5,
    legend=dict(
        yanchor="top",
        y=1,
        xanchor="right",
        x=0.9
    )
)
pcr_fig.show()

Make a chart of the term structure using the implied volatility skew, or the cost of an at-the-money straddle.

In [22]:
skew_df = pd.DataFrame()

ticker = openbb.stocks.options.load_options_chains("QQQ")
skew = ticker.get_skew("2023-12-15")
call_skew = skew[skew["Option Type"] == "call"].set_index("Strike")[["Skew"]]
put_skew = skew[skew["Option Type"] == "put"].set_index("Strike")[["Skew"]]
skew_df["Call Skew"] = call_skew["Skew"]
skew_df["Put Skew"] = put_skew["Skew"]

openbb.forecast.plot(skew_df, columns = skew_df.columns.to_list())

In [23]:
skew = ticker.get_skew(moneyness=10)[["ATM Skew", "IV Skew"]].rename(columns = {"IV Skew": "10% OTM Skew"})
y1 = pd.DataFrame(skew["ATM Skew"])
y2 = pd.DataFrame(skew["10% OTM Skew"])
skew_fig = show_plot(y1,y2, external_axes = True)
skew_fig.update_layout(
    xaxis = {'title': 'Expiration Date'},
    yaxis= {'title': 'ATM IV Skew'},
    yaxis2 = {'title': '10% OTM IV Skew'},
    title =  f"{ticker.symbol} - ATM Skew vs 10% OTM Skew",
    title_y = 0.98,
    title_x = 0.5,
    legend=dict(
        yanchor="bottom",
        y=0.1,
        xanchor="right",
        x=0.9
    )
)
skew_fig.update_yaxes(title_font=dict(size=20), tickfont=dict(size=12))
skew_fig.show()

In [24]:
y1 = pd.DataFrame(ticker.get_strategies().set_index("Expiration")["Cost Percent"])
y2 = pd.DataFrame(skew[["10% OTM Skew"]])
skew_fig2 = show_plot(y1,y2, external_axes = True)
skew_fig2.update_layout(
    xaxis = {'title': 'Expiration Date'},
    yaxis= {'title': 'Cost as % of Stock Price'},
    yaxis2 = {'title': '10% OTM IV Skew'},
    title =  f"{ticker.symbol} - Cost of ATM Straddle vs 10% OTM Skew",
    title_y = 0.98,
    title_x = 0.5,
    legend=dict(
        yanchor="bottom",
        y=0.1,
        xanchor="right",
        x=0.9
    )
)
skew_fig2.update_yaxes(title_font=dict(size=16))
skew_fig2.show()

No strategy was selected, returning all ATM straddles.


Basic single-leg strategies can be screened via `get_strategies()`.  Values entered for each argument return the closest match.  With no parameters specified, all ATM straddles are returned.

In [25]:
ticker.get_strategies(days = [5,30], straddle_strike = ticker.last_price, strangle_moneyness = [10,-20] , vertical_calls = [365,367], vertical_puts = [360,358])

,Expiration,DTE,Strategy,Underlying Price,Strike 1,Strike 2,Strike 1 Premium,Strike 2 Premium,Cost,Cost Percent,Breakeven Upper,Breakeven Upper Percent,Breakeven Lower,Breakeven Lower Percent,Max Profit,Max Loss,Payoff Ratio
0,2023-06-28,5,Long Straddle,364.05,364.0,364.0,2.79,2.65,5.44,1.4943,369.44,1.4806,358.56,-1.508,inf,-5.44,inf
1,2023-06-28,5,Long Strangle,364.05,400.0,330.0,0.02,0.03,0.05,0.0137,400.05,9.8888,329.95,-9.3668,inf,-0.05,inf
2,2023-06-28,5,Bear Call Spread,364.05,365.0,367.0,2.28,1.51,-0.77,-0.2115,365.77,0.5988,NaN,NaN,0.77,-1.23,0.626
3,2023-06-28,5,Bull Put Spread,364.05,360.0,358.0,1.2,0.8,-0.4,0.1099,359.6,1.2224,NaN,NaN,0.4,-1.6,0.25
4,2023-07-21,28,Long Straddle,364.05,364.0,364.0,7.72,6.37,14.09,3.8703,378.09,3.8566,349.91,-3.8841,inf,-14.09,inf
5,2023-07-21,28,Long Strangle,364.05,400.0,327.0,0.19,0.51,0.7,0.1923,400.7,10.0673,326.3,-10.3695,inf,-0.7,inf
6,2023-07-21,28,Short Strangle,364.05,435.0,291.0,0.01,0.07,-0.08,0.022,435.08,19.5111,290.92,-20.0879,0.08,inf,0.0
7,2023-07-21,28,Bear Call Spread,364.05,365.0,367.0,7.14,6.12,-1.02,-0.2802,366.02,0.5301,NaN,NaN,1.02,-0.98,1.0408
8,2023-07-21,28,Bull Put Spread,364.05,360.0,358.0,4.8,4.18,-0.62,0.1703,359.38,1.2828,NaN,NaN,0.62,-1.38,0.4493


In [26]:
ticker.get_strategies()

No strategy was selected, returning all ATM straddles.


,Expiration,DTE,Strategy,Underlying Price,Strike 1,Strike 2,Strike 1 Premium,Strike 2 Premium,Cost,Cost Percent,Breakeven Upper,Breakeven Upper Percent,Breakeven Lower,Breakeven Lower Percent,Max Profit,Max Loss,Payoff Ratio
0,2023-06-26,3,Long Straddle,364.05,364.0,364.0,1.76,1.71,3.47,0.9532,367.47,0.9394,360.53,-0.9669,inf,-3.47,inf
1,2023-06-27,4,Long Straddle,364.05,364.0,364.0,2.31,2.21,4.52,1.2416,368.52,1.2279,359.48,-1.2553,inf,-4.52,inf
2,2023-06-28,5,Long Straddle,364.05,364.0,364.0,2.79,2.65,5.44,1.4943,369.44,1.4806,358.56,-1.508,inf,-5.44,inf
3,2023-06-29,6,Long Straddle,364.05,364.0,364.0,3.23,2.95,6.18,1.6976,370.18,1.6838,357.82,-1.7113,inf,-6.18,inf
4,2023-06-30,7,Long Straddle,364.05,364.0,364.0,3.66,3.28,6.94,1.9063,370.94,1.8926,357.06,-1.9201,inf,-6.94,inf
5,2023-07-03,10,Long Straddle,364.05,364.0,364.0,4.0,3.57,7.57,2.0794,371.57,2.0657,356.43,-2.0931,inf,-7.57,inf
6,2023-07-05,12,Long Straddle,364.05,364.0,364.0,4.41,3.92,8.33,2.2881,372.33,2.2744,355.67,-2.3019,inf,-8.33,inf
7,2023-07-06,13,Long Straddle,364.05,364.0,364.0,4.77,4.15,8.92,2.4502,372.92,2.4365,355.08,-2.4639,inf,-8.92,inf
8,2023-07-07,14,Long Straddle,364.05,364.0,364.0,5.15,4.46,9.61,2.6397,373.61,2.626,354.39,-2.6535,inf,-9.61,inf
9,2023-07-14,21,Long Straddle,364.05,364.0,364.0,6.54,5.52,12.06,3.3127,376.06,3.299,351.94,-3.3265,inf,-12.06,inf


It's now easier than ever to analyze equity options.  See the introduction guide [here](https://docs.openbb.co/sdk/usage/intros/stocks/options-chains) for more information on using these functions.